In [1]:
# Imports
import requests
import matplotlib.pyplot as plt
import numpy as np 
from pathlib import Path 
import json
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv
import csv
from bs4 import BeautifulSoup
from sklearn import linear_model 
from sklearn import svm
from sklearn import ensemble
import warnings 
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import warnings 

In [2]:
warnings.filterwarnings('ignore')

In [24]:
#read_csv file for "FF_qb_2020.csv" and "FF_qb_2021.csv"
qb_data_2020 = pd.read_csv("FF_qb_2020.csv")
qb_data_2021 = pd.read_csv("FF_qb_2021.csv")
qb_data_2020

,Rank,Player,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT.1,YDS.1,TD.1,FL,G,FPTS,FPTS/G,ROST
0,1,Josh Allen (BUF),396,572,69.2,"4,544",7.9,37,10,26,102,421,8,6,16,405.2,25.3,100.00%
1,2,Kyler Murray (ARI),375,558,67.2,"3,971",7.1,26,12,27,133,819,11,4,16,390.7,24.4,98.50%
2,3,Aaron Rodgers (GB),372,526,70.7,"4,299",8.2,48,5,20,38,149,3,2,16,387.1,24.2,96.00%
3,4,Patrick Mahomes II (KC),390,588,66.3,"4,740",8.1,38,6,22,62,308,2,2,15,380.3,25.4,100.00%
4,5,Deshaun Watson (CLE),382,544,70.2,"4,823",8.9,33,7,49,90,444,3,3,16,376.4,23.5,15.80%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,81,Easton Stick (LAC),1,1,100.0,4,4.0,0,0,0,1,-2,0,0,1,0.0,0.0,0.00%
81,82,Mike White (NYJ),0,0,0.0,0,0.0,0,0,0,0,0,0,0,1,0.0,0.0,0.00%
82,83,Matt Schaub (FA),0,0,0.0,0,0.0,0,0,0,3,-4,0,0,1,-0.4,-0.4,0.00%
83,84,Nate Sudfeld (DET),5,12,41.7,32,2.7,0,1,2,2,12,0,1,1,-0.5,-0.5,0.00%


In [25]:
#Get column data for Player, Fantasy Points, and Fantasay points a game. dropna. 
qb_data_2020 = qb_data_2020[["Player" , "FPTS", "FPTS/G"]].dropna()
qb_data_2021 = qb_data_2021[["Player" , "FPTS", "FPTS/G"]].dropna()

In [26]:
#Add years and QB's to the table names using add_prefix function
#qb_data_2020 = qb_data_2020.add_prefix("2020_QBs_")
#qb_data_2021 = qb_data_2021.add_prefix("2021_QBs_")

In [27]:
#Display data
qb_data_2020

,Player,FPTS,FPTS/G
0,Josh Allen (BUF),405.2,25.3
1,Kyler Murray (ARI),390.7,24.4
2,Aaron Rodgers (GB),387.1,24.2
3,Patrick Mahomes II (KC),380.3,25.4
4,Deshaun Watson (CLE),376.4,23.5
...,...,...,...
80,Easton Stick (LAC),0.0,0.0
81,Mike White (NYJ),0.0,0.0
82,Matt Schaub (FA),-0.4,-0.4
83,Nate Sudfeld (DET),-0.5,-0.5


In [30]:
#Get column data for Player, Fantasy Points, and Fantasay points a game. dropna. 
qb_data_2020 = qb_data_2020[["Player", "FPTS", "FPTS/G",]].dropna()
qb_data_2021 = qb_data_2021[["Player", "FPTS", "FPTS/G",]].dropna()

qb_data_2020["Year"] = 2020
qb_data_2021["Year"] = 2021

#Display
display(qb_data_2020)

#Display
display(qb_data_2021)

,Player,FPTS,FPTS/G,Year
0,Josh Allen (BUF),405.2,25.3,2020
1,Kyler Murray (ARI),390.7,24.4,2020
2,Aaron Rodgers (GB),387.1,24.2,2020
3,Patrick Mahomes II (KC),380.3,25.4,2020
4,Deshaun Watson (CLE),376.4,23.5,2020
...,...,...,...,...
80,Easton Stick (LAC),0.0,0.0,2020
81,Mike White (NYJ),0.0,0.0,2020
82,Matt Schaub (FA),-0.4,-0.4,2020
83,Nate Sudfeld (DET),-0.5,-0.5,2020


,Player,FPTS,FPTS/G,Year
0,Josh Allen (BUF),417.7,24.6,2021
1,Justin Herbert (LAC),395.6,23.3,2021
2,Tom Brady (TB),386.7,22.7,2021
3,Patrick Mahomes II (KC),374.2,22.0,2021
4,Matthew Stafford (LAR),346.8,20.4,2021
...,...,...,...,...
117,Davis Webb (NYG),-0.3,-0.3,2021
118,Feleipe Franks (ATL),-0.4,-0.1,2021
119,Logan Woodside (TEN),-0.6,-0.2,2021
120,John Wolford (LAR),-0.9,-0.5,2021


In [31]:
#qb_data_2020.insert(loc=1, column="Fantasy Pos", value=["QB"])
qb_data_2020["Fantasy Pos"]= "QB"

qb_data_2021["Fantasy Pos"]= "QB"

In [32]:
#rename qb_data_2020 and qb_data_2021 columns. 
qb_data_2020.rename(columns={
    "FPTS":"Fantasy Points",
    "FPTS/G":"Fantasy Points/GM"}, inplace=True)


qb_data_2021.rename(columns={
    "FPTS":"Fantasy Points",
    "FPTS/G":"Fantasy Points/GM"}, inplace=True)

#Display
display(qb_data_2020)
#Dispaly 
display(qb_data_2021)

,Player,Fantasy Points,Fantasy Points/GM,Year,Fantasy Pos
0,Josh Allen (BUF),405.2,25.3,2020,QB
1,Kyler Murray (ARI),390.7,24.4,2020,QB
2,Aaron Rodgers (GB),387.1,24.2,2020,QB
3,Patrick Mahomes II (KC),380.3,25.4,2020,QB
4,Deshaun Watson (CLE),376.4,23.5,2020,QB
...,...,...,...,...,...
80,Easton Stick (LAC),0.0,0.0,2020,QB
81,Mike White (NYJ),0.0,0.0,2020,QB
82,Matt Schaub (FA),-0.4,-0.4,2020,QB
83,Nate Sudfeld (DET),-0.5,-0.5,2020,QB


,Player,Fantasy Points,Fantasy Points/GM,Year,Fantasy Pos
0,Josh Allen (BUF),417.7,24.6,2021,QB
1,Justin Herbert (LAC),395.6,23.3,2021,QB
2,Tom Brady (TB),386.7,22.7,2021,QB
3,Patrick Mahomes II (KC),374.2,22.0,2021,QB
4,Matthew Stafford (LAR),346.8,20.4,2021,QB
...,...,...,...,...,...
117,Davis Webb (NYG),-0.3,-0.3,2021,QB
118,Feleipe Franks (ATL),-0.4,-0.1,2021,QB
119,Logan Woodside (TEN),-0.6,-0.2,2021,QB
120,John Wolford (LAR),-0.9,-0.5,2021,QB


In [33]:
#move column name "Fantasy Pos" to 2nd column position.
column_name_move = qb_data_2020.pop("Fantasy Pos")

qb_data_2020.insert(1, "Fantasy Pos", column_name_move)


column_name_move = qb_data_2021.pop("Fantasy Pos")

qb_data_2021.insert(1, "Fantasy Pos", column_name_move)

#Display
display(qb_data_2020)

#Display
display(qb_data_2021)

,Player,Fantasy Pos,Fantasy Points,Fantasy Points/GM,Year
0,Josh Allen (BUF),QB,405.2,25.3,2020
1,Kyler Murray (ARI),QB,390.7,24.4,2020
2,Aaron Rodgers (GB),QB,387.1,24.2,2020
3,Patrick Mahomes II (KC),QB,380.3,25.4,2020
4,Deshaun Watson (CLE),QB,376.4,23.5,2020
...,...,...,...,...,...
80,Easton Stick (LAC),QB,0.0,0.0,2020
81,Mike White (NYJ),QB,0.0,0.0,2020
82,Matt Schaub (FA),QB,-0.4,-0.4,2020
83,Nate Sudfeld (DET),QB,-0.5,-0.5,2020


,Player,Fantasy Pos,Fantasy Points,Fantasy Points/GM,Year
0,Josh Allen (BUF),QB,417.7,24.6,2021
1,Justin Herbert (LAC),QB,395.6,23.3,2021
2,Tom Brady (TB),QB,386.7,22.7,2021
3,Patrick Mahomes II (KC),QB,374.2,22.0,2021
4,Matthew Stafford (LAR),QB,346.8,20.4,2021
...,...,...,...,...,...
117,Davis Webb (NYG),QB,-0.3,-0.3,2021
118,Feleipe Franks (ATL),QB,-0.4,-0.1,2021
119,Logan Woodside (TEN),QB,-0.6,-0.2,2021
120,John Wolford (LAR),QB,-0.9,-0.5,2021


In [95]:
#concat qb_dataframes and dropna. 
combine_qb_data= pd.concat([qb_data_2020, qb_data_2021], axis=0, sort=False).dropna()
combine_qb_data.reset_index(inplace=True, drop=True)
combine_qb_data

,Player,Fantasy Pos,Fantasy Points,Fantasy Points/GM,Year
0,Josh Allen (BUF),QB,405.2,25.3,2020
1,Kyler Murray (ARI),QB,390.7,24.4,2020
2,Aaron Rodgers (GB),QB,387.1,24.2,2020
3,Patrick Mahomes II (KC),QB,380.3,25.4,2020
4,Deshaun Watson (CLE),QB,376.4,23.5,2020
...,...,...,...,...,...
202,Davis Webb (NYG),QB,-0.3,-0.3,2021
203,Feleipe Franks (ATL),QB,-0.4,-0.1,2021
204,Logan Woodside (TEN),QB,-0.6,-0.2,2021
205,John Wolford (LAR),QB,-0.9,-0.5,2021


In [96]:
#create a team list and player list 
team_list = []
player_name_list = []

# create a forloop to split the player name and player team. 
for i in range(0,len(combine_qb_data["Player"])):
    team_list.append(combine_qb_data["Player"][i].split()[-1].replace("(","").replace(")",""))
    player_name_list.append(" ".join(combine_qb_data["Player"][i].split()[:-1]))

#create player 
combine_qb_data["Player"] = player_name_list
combine_qb_data["Team"] = team_list
combine_qb_data

,Player,Fantasy Pos,Fantasy Points,Fantasy Points/GM,Year,Team
0,Josh Allen,QB,405.2,25.3,2020,BUF
1,Kyler Murray,QB,390.7,24.4,2020,ARI
2,Aaron Rodgers,QB,387.1,24.2,2020,GB
3,Patrick Mahomes II,QB,380.3,25.4,2020,KC
4,Deshaun Watson,QB,376.4,23.5,2020,CLE
...,...,...,...,...,...,...
202,Davis Webb,QB,-0.3,-0.3,2021,NYG
203,Feleipe Franks,QB,-0.4,-0.1,2021,ATL
204,Logan Woodside,QB,-0.6,-0.2,2021,TEN
205,John Wolford,QB,-0.9,-0.5,2021,LAR


In [108]:
combine_qb_data.groupby(["Player"]).mean()[["Fantasy Points", "Fantasy Points/GM"]].sort_values("Fantasy Points/GM",ascending=False).reset_index()

,Player,Fantasy Points,Fantasy Points/GM
0,Josh Allen,411.45,24.95
1,Dak Prescott,234.50,24.20
2,Patrick Mahomes II,377.25,23.70
3,Kyler Murray,350.60,23.30
4,Justin Herbert,369.20,23.10
...,...,...,...
129,Feleipe Franks,-0.40,-0.10
130,Nate Sudfeld,-0.25,-0.25
131,Davis Webb,-0.30,-0.30
132,Josh Rosen,-1.20,-0.40


In [63]:
import hvplot.pandas
combine_qb_data[combine_qb_data["Year"]==2020].hvplot.scatter(y="Fantasy Points", x="Fantasy Points/GM", by="Year", hover_cols=["Player"])

:NdOverlay   [Year]
   :Scatter   [Fantasy Points/GM]   (Fantasy Points,Player)

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
train, test= train_test_split(combine_qb_data, test_size=0.2, random_state=1) 

In [41]:
print(train.head())

                 Player Fantasy Pos  Fantasy Points  Fantasy Points/GM  Year
73    Case Keenum (BUF)          QB             1.8                0.9  2020
166    Brock Purdy (SF)          QB             0.0                0.0  2021
138     PJ Walker (CAR)          QB            16.8                3.4  2021
199  Jeff Driskel (HOU)          QB             0.0                0.0  2021
84      Tim Boyle (DET)          QB            -0.9               -0.1  2020


In [42]:
print(test.head())

                    Player Fantasy Pos  Fantasy Points  Fantasy Points/GM  \
186      Ryan Griffin (TB)          QB             0.0                0.0   
155  Ryan Fitzpatrick (FA)          QB             0.7                0.7   
165    Brett Ratliff (TEN)          QB             0.0                0.0   
200      Kurt Benkert (FA)          QB            -0.1               -0.1   
58         PJ Walker (CAR)          QB            13.5                3.4   

     Year  
186  2021  
155  2021  
165  2021  
200  2021  
58   2020  


In [44]:
print(train.shape, test.shape)

(165, 5) (42, 5)
